In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import joblib
from torchvision.models import resnet50, ResNet50_Weights
from torchvision import models

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=1),
    transforms.RandomRotation(degrees=10),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    transforms.RandomResizedCrop(size=(224, 224), scale=(0.9, 1.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

data_dir = r'C:\Users\aaa\Documents\DATASETS\Organized_Amharic_Character_Dataset'
dataset = ImageFolder(root=data_dir, transform=transform)
targets = np.array(dataset.targets)

train_indices, val_indices = train_test_split(range(len(dataset)), test_size=0.2, stratify=targets, random_state=42)
train_subset = Subset(dataset, train_indices)
val_subset = Subset(dataset, val_indices)

train_loader = DataLoader(train_subset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=64, shuffle=False)

for images, labels in train_loader:
    print(f"Sample batch shape: {images.shape}")
    break

Sample batch shape: torch.Size([64, 1, 224, 224])


In [ ]:
#VGG19
import torch
import torch.nn as nn
import numpy as np
from torchvision import models
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import joblib

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vgg19_base = models.vgg19(weights='DEFAULT')
vgg19_base.features[0] = nn.Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
vgg19_base.classifier = nn.Sequential(*list(vgg19_base.classifier.children())[:-1])

class CustomVGG19(nn.Module):
    def __init__(self, base_model, num_classes):
        super(CustomVGG19, self).__init__()
        self.base_model = base_model
        self.dropout = nn.Dropout(p=0.3)
        self.fc = nn.Linear(4096, num_classes)

    def forward(self, x):
        x = self.base_model(x)
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        x = self.fc(x)
        return x

num_classes = 238
model_2 = CustomVGG19(vgg19_base, num_classes).to(device)

model_2.load_state_dict(torch.load(r'C:\Users\aaa\RESEARCH\model_2_weights.pth', weights_only=True))

def extract_features(dataloader, model):
    model.eval()
    features = []
    labels = []
    with torch.no_grad():
        for i, (inputs, targets) in enumerate(dataloader):
            inputs = inputs.to(device)
            outputs = model(inputs)
            features.append(outputs.cpu().numpy())
            labels.append(targets.numpy())
            if i % 10 == 0:  # Track progress every 10 batches
                print(f"Extracted features for batch {i+1}")
    return np.concatenate(features), np.concatenate(labels)

print("Step 1: Extracting training features...")
train_features, train_labels = extract_features(train_loader, model_2)
print("Finished extracting training features.")

print("Step 2: Extracting validation features...")
val_features, val_labels = extract_features(val_loader, model_2)
print("Finished extracting validation features.")
#VGG19+RANDOM FOREST
print("Step 3: Training Random Forest Classifier...")
rf_classifier = RandomForestClassifier(n_estimators=200, random_state=42)
rf_classifier.fit(train_features, train_labels)
print("Finished training Random Forest Classifier.")

def evaluate_in_batches(dataloader, model, classifier):
    model.eval()
    batch_acc = []
    batch_prec = []
    batch_rec = []
    batch_f1 = []

    with torch.no_grad():
        for inputs, targets in dataloader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            preds = classifier.predict(outputs.cpu().numpy())

            acc = accuracy_score(targets.numpy(), preds)
            prec = precision_score(targets.numpy(), preds, average='weighted', zero_division=0)
            rec = recall_score(targets.numpy(), preds, average='weighted', zero_division=0)
            f1 = f1_score(targets.numpy(), preds, average='weighted', zero_division=0)

            batch_acc.append(acc)
            batch_prec.append(prec)
            batch_rec.append(rec)
            batch_f1.append(f1)

    avg_acc = np.mean(batch_acc)
    avg_prec = np.mean(batch_prec)
    avg_rec = np.mean(batch_rec)
    avg_f1 = np.mean(batch_f1)

    return avg_acc, avg_prec, avg_rec, avg_f1

print("Step 4: Evaluating on training data...")
train_avg_acc, train_avg_prec, train_avg_rec, train_avg_f1 = evaluate_in_batches(train_loader, model_2, rf_classifier)

print(f"Training Accuracy: {train_avg_acc:.4f}")
print(f"Training Precision: {train_avg_prec:.4f}")
print(f"Training Recall: {train_avg_rec:.4f}")
print(f"Training F1 Score: {train_avg_f1:.4f}")

print("Step 5: Evaluating on validation data...")
val_avg_acc, val_avg_prec, val_avg_rec, val_avg_f1 = evaluate_in_batches(val_loader, model_2, rf_classifier)

print(f"Validation Accuracy: {val_avg_acc:.4f}")
print(f"Validation Precision: {val_avg_prec:.4f}")
print(f"Validation Recall: {val_avg_rec:.4f}")
print(f"Validation F1 Score: {val_avg_f1:.4f}")

joblib.dump(rf_classifier, 'rf_model_vgg19.pkl')
print("Model saved as rf_model_vgg19.pkl")

print("All steps completed!")

Step 1: Extracting training features...
Extracted features for batch 1
Extracted features for batch 11
Extracted features for batch 21
Extracted features for batch 31
Extracted features for batch 41
Extracted features for batch 51
Extracted features for batch 61
Extracted features for batch 71
Extracted features for batch 81
Extracted features for batch 91
Extracted features for batch 101
Extracted features for batch 111
Extracted features for batch 121
Extracted features for batch 131
Extracted features for batch 141
Extracted features for batch 151
Extracted features for batch 161
Extracted features for batch 171
Extracted features for batch 181
Extracted features for batch 191
Extracted features for batch 201
Extracted features for batch 211
Extracted features for batch 221
Extracted features for batch 231
Extracted features for batch 241
Extracted features for batch 251
Extracted features for batch 261
Extracted features for batch 271
Extracted features for batch 281
Extracted feat

In [ ]:
#VGG-19+XGBOOST
import numpy as np
from xgboost import XGBClassifier, DMatrix
import joblib

print("Training XGBoost Classifier...")
xgb_classifier = XGBClassifier(
    eval_metric='mlogloss',
    n_estimators=200,
    random_state=42,
    tree_method='hist',
    device='cuda'
)
xgb_classifier.fit(train_features, train_labels)
print("Finished training XGBoost Classifier.")

print("Evaluating on training data...")
train_avg_acc, train_avg_prec, train_avg_rec, train_avg_f1 = evaluate_in_batches(train_loader, model_2, xgb_classifier)

print(f"Training Accuracy: {train_avg_acc:.4f}")
print(f"Training Precision: {train_avg_prec:.4f}")
print(f"Training Recall: {train_avg_rec:.4f}")
print(f"Training F1 Score: {train_avg_f1:.4f}")

print("Evaluating on validation data...")
val_avg_acc, val_avg_prec, val_avg_rec, val_avg_f1 = evaluate_in_batches(val_loader, model_2, xgb_classifier)

print(f"Validation Accuracy: {val_avg_acc:.4f}")
print(f"Validation Precision: {val_avg_prec:.4f}")
print(f"Validation Recall: {val_avg_rec:.4f}")
print(f"Validation F1 Score: {val_avg_f1:.4f}")

joblib.dump(xgb_classifier, 'xgb_model_vgg19.pkl')
print("Model saved as xgb_model_vgg19.pkl")

print("All steps completed!")

Training XGBoost Classifier...
Finished training XGBoost Classifier.
Evaluating on training data...
Training Accuracy: 0.9526
Training Precision: 0.9572
Training Recall: 0.9526
Training F1 Score: 0.9525
Evaluating on validation data...
Validation Accuracy: 0.8817
Validation Precision: 0.8943
Validation Recall: 0.8817
Validation F1 Score: 0.8824
Model saved as xgb_model_vgg19.pkl
All steps completed!


In [ ]:
#VGG-19+SVM
import numpy as np
from sklearn.svm import SVC
import joblib

print("Training SVM Classifier...")
svm_classifier = SVC(kernel='rbf', random_state=42, gamma='scale')
svm_classifier.fit(train_features, train_labels)
print("Finished training SVM Classifier.")

print("Evaluating on training data...")
train_avg_acc, train_avg_prec, train_avg_rec, train_avg_f1 = evaluate_in_batches(train_loader, model_2, svm_classifier)

print(f"Training Accuracy: {train_avg_acc:.4f}")
print(f"Training Precision: {train_avg_prec:.4f}")
print(f"Training Recall: {train_avg_rec:.4f}")
print(f"Training F1 Score: {train_avg_f1:.4f}")

print("Evaluating on validation data...")
val_avg_acc, val_avg_prec, val_avg_rec, val_avg_f1 = evaluate_in_batches(val_loader, model_2, svm_classifier)

print(f"Validation Accuracy: {val_avg_acc:.4f}")
print(f"Validation Precision: {val_avg_prec:.4f}")
print(f"Validation Recall: {val_avg_rec:.4f}")
print(f"Validation F1 Score: {val_avg_f1:.4f}")

joblib.dump(svm_classifier, 'svm_model_vgg19.pkl')
print("Model saved as svm_model_vgg19.pkl")

print("All steps completed!")

Training SVM Classifier...
Finished training SVM Classifier.
Evaluating on training data...
Training Accuracy: 0.9601
Training Precision: 0.9642
Training Recall: 0.9601
Training F1 Score: 0.9600
Evaluating on validation data...
Validation Accuracy: 0.9017
Validation Precision: 0.9097
Validation Recall: 0.9017
Validation F1 Score: 0.9011
Model saved as svm_model_vgg19.pkl
All steps completed!
